# Desafío de data science Itti
![title](../img/itti.png)

In [ ]:
# Maximiliano Angel Uboldi

## 1 - SQL

Para el siguiente ejercicio se tiene una base de datos que tiene el siguiente DER:

![title](../img/img_der.png)


* Persona (id) puede tener muchas Tarjetas (se relaciona a través de `id_titular`)
* Una Compra siempre es realizada con una única tarjeta
* (EXTRA) La tabla Compra posee un histórico de más de 10 años y varias decenas de millones de registros por lo que las consultas a esta entidad pueden tardar varios minutos si no están correctamente optimizadas.

### sql-1) Query

Obtener los número de tarjeta que no tengan consumos en el último mes pero que pertenecen a personas que hayan comprado al menos un total de $5000 en el último mes en los rubros (FARMACIA y SUPERMERCADOS) en conjunto. (Si es posible, tener en cuenta el punto EXTRA mencionado previamente).

In [ ]:
-- Reducimos el número de compras en una CTE a las realizadas en el último mes para optimizar el rendimiento (dependiendo del motor de base de datos, que no sabemos cuál es).
WITH last_month_purchases AS (
    SELECT *
    FROM Compra
    WHERE fecha_compra >= DATEADD(MONTH, -1, CURRENT_DATE)
)
-- Seleccionamos las personas que cumplan con los criterios de compra en farmacia y supermercados (ambos rubros), y al menos 5000 en total.
,qualified_persons AS (
    SELECT
        p.id
    FROM Persona p
    JOIN Tarjeta t
        ON p.id = t.id_titular
    JOIN last_month_purchases c
        ON t.nro_tarjeta = c.nro_tarjeta
    WHERE
        c.rubro IN ('FARMACIA', 'SUPERMERCADOS')
    GROUP BY
        p.id
    HAVING
        SUM(c.monto) >= 5000
        AND COUNT(DISTINCT c.rubro) = 2
)
-- Seleccionamos las tarjetas con las que las personas que cumplieron los criterios anteriores no realizaron compras en el último mes.
SELECT
    t.nro_tarjeta
FROM Tarjeta t
JOIN qualified_persons p
    ON t.id_titular = p.id
WHERE
    NOT EXISTS (
    SELECT 1
    FROM last_month_purchases c
    WHERE c.nro_tarjeta = t.nro_tarjeta
)
;

## 2- Analysis geoespacial 

Se cuenta con el dataset ubicaciones.csv, dicho dataset posee coordenadas de diferentes lugares de interes.
<br>Se sabe que cuando se agrupan 8 lugares o mas esa zona se la considera zona comercial.
<br>Para decir que un local pertenece a la misma zona que otro, deben estar a una distancia menor a 1.2 km
- Crear poligonos con todas las zonas comerciales que se encuentren en el dataset.
- Una vez encontradas marcar su zona de influencia que es dentro del poligono pero tambien hasta 400m de los limites del mismo
- Graficar en un mapa todos los comercios y las zonas de influencia comercial

In [ ]:
import folium
import geopandas as gpd
import pandas as pd
from shapely.geometry import mapping
from sklearn.cluster import DBSCAN

In [ ]:
df = pd.read_csv("../data/ubicaciones.csv")

In [ ]:
df

In [ ]:
gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df["longitud"], df["latitud"]),
    crs="EPSG:4326" # Longitude/Latitude
)

In [ ]:
gdf = gdf.to_crs(epsg=3857) # Web Mercator, meters

In [ ]:
coords = list(zip(gdf.geometry.x, gdf.geometry.y))
db = DBSCAN(eps=1200, min_samples=8, metric="euclidean").fit(coords)
gdf["cluster"] = db.labels_

In [ ]:
gdf

In [ ]:
commercial = gdf[gdf['cluster'] != -1].copy()
clusters = commercial['cluster'].unique()

In [ ]:
polygons = []
areas_influence = []
for cl in clusters:
    puntos = commercial[commercial['cluster'] == cl].geometry
    polygon = puntos.union_all().convex_hull
    polygons.append(polygon)
    area_inf = polygon.buffer(400)
    areas_influence.append(area_inf)

In [ ]:
gdf_polygons = gpd.GeoDataFrame(
    {'geometry': polygons}, crs='EPSG:3857').to_crs(epsg=4326)
gdf_influence = gpd.GeoDataFrame(
    {'geometry': areas_influence}, crs='EPSG:3857').to_crs(epsg=4326)
gdf = gdf.to_crs(epsg=4326)

In [ ]:
center = [gdf.geometry.y.mean(), gdf.geometry.x.mean()]
map_var = folium.Map(location=center, zoom_start=10)

In [ ]:
for _, row in gdf.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=3,
        color='grey',
        fill=True,
        fill_opacity=0.7
    ).add_to(map_var)

In [ ]:
for _, row in gdf_influence.iterrows():
    folium.GeoJson(
        mapping(row.geometry),
        style_function=lambda x: {
            'color': 'red', 'weight': 2, 'fill': False, 'dashArray': '5, 5'},
        name='Zona de Influencia'
    ).add_to(map_var)

In [ ]:
for _, row in gdf_polygons.iterrows():
    folium.GeoJson(
        mapping(row.geometry),
        style_function=lambda x: {'color': 'blue', 'weight': 2, 'fill': False},
        name='Zona Comercial'
    ).add_to(map_var)

In [ ]:
folium.LayerControl().add_to(map_var)

In [ ]:
map_var

## 3- Machine Learning

# Problema de Retención de Clientes en el Banco PY

Se desea resolver el siguiente problema: 

El banco PY está notando que muchos clientes se están acercando a cerrar su cuenta, lo cual les preocupa. Decidieron implementar una estrategia de retención muy agresiva que, a su vez, es costosa ya que bonifica productos y otorga descuentos que no se ven normalmente en el mercado. 

Se ha observado en un pequeño experimento que, a pesar de ser una oferta irresistible, cuando un cliente le indica al banco que se quiere dar de baja, ya no hay vuelta atrás. Por esta razón, se desea que los ejecutivos sepan con anticipación cuándo un cliente tiene altas probabilidades de pedir la baja, a pesar de que el motivo de acercarse al banco sea otro. De esta manera, al recibir al cliente, ya se pueden preparar para retenerlos.

## Descripción Operativa

Cuando un cliente entra en una sucursal, tiene un tótem en donde se registra e indica el motivo de su visita. Esto le genera el turno y, con él, espera a ser atendido por un ejecutivo. 

El banco posee varias sucursales en Asunción que se gestionan en dos zonas: una céntrica y otra en las afueras. Esta definición no solo tiene que ver con la locación, sino también con el comportamiento y la tolerancia de los clientes.

Hoy en día, el equipo de Data Science tiene funcionando un modelo, pero no es suficientemente bueno para la nueva estrategia que propone el equipo comercial, por lo que desean reemplazarlo por uno nuevo.

## Objetivos

* Desarrollar un modelo de Machine Learning que prediga churn.
* Definir cómo sería la implementación y cuándo se usaría.
* Explicar la elección del algoritmo utilizado
* Explicar los resultados obtenidos (Métricas) y si tiene el rendimiento necesario para salir a producción.
* Explicar cuales son las variables que más afectan a la predicción.
* Contestar las siguientes preguntas:
  * *¿El modelo final elegido es bueno? Justificar*
  * *¿Existe overfitting o underfitting?* 

⚠️**ACLARACIONES:**⚠️

- Se tendrá en cuenta la prolijidad del informe. 
- Se tendrá en cuenta la legibilidad del código.
- De ser necesario, dejar asentado los supuestos utilizados por escrito.

## Recursos Disponibles

Para resolver el desafío, se cuenta con:

- Un dataset que contiene las visitas de nuestros clientes a las sucursales y otro de zonas. Esta información consta de datos obtenidos del tótem y del CRM del ejecutivo.
- Un modelo actual (función de Python para hacer predicciones).
- Datos de costo promedio de la estrategia de retención y rentabilidad esperada del cliente en los 3 primeros meses luego de retenerlo, rentabilidad esperada del cliente en el primer año luego de retenerlo, y rentabilidad del cliente en la última acción de darse de baja.

## Descripción del dataset
- cliente_id: id del cliente que se registra en el totem de la sucursal
- segmento_cliente: segmento del cliente, se obtiene con un modelo de ML que segmenta a los clientes según su comportamiento
- tipo_asistencia: la accion que declara el cliente que viene a hacer al banco cuando se registra en el totem. En los últimos meses de volvió opcional
- descripcion_atencion: cuando el cliente se registra le dice al totem con su voz qué es lo que viene a hacer al banco, un modelo de IA lo traduce en tiempo real. El cliente también tiene la opción de hacerlo escribiendo en un teclado.
- coordenadas_sucursal: coordenadas geográficas en donde se encuentra la sucursal de la atención
- inicio_atencion_utc: fecha y hora de inicio de llegada a la sucursa (registro en totem). El huso horario es UTC+0
- fin_atencion: fecha y hora de fin de atención marcada desde el CRM del ejecutivo cuando cierra la atención. El huso horario es UTC-3 hora de Paraguay
- puntos_de_loyalty: puntos acumulados en el sistema de loyalty
- edad: edad del cliente
- churn: indica si luego de la atención el cliente se dio de baja del banco o no. Se completa automáticamente desde el CRM por el ejecutivo.

- zona: nombre de la zona de atención
- poligono: coordenadas del contorno del polígono correspondiente a cada zona

In [ ]:
import ast
import gc
import json
import random
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, Polygon
import seaborn as sns
from matplotlib.ticker import StrMethodFormatter

In [ ]:
df = pd.read_csv(
    "../data/dataset_churn_challenge.csv",
    dtype={
        "cliente_id": "object",
        "segmento_cliente": "object",
        "tipo_asistencia": "object",
        "descripcion_atencion": "object",
        "coordenadas_sucursal": "object",
        "inicio_atencion_utc": "object",
        "fin_atencion": "object",
        "puntos_de_loyalty": "float64",
        "edad": "int64",
        "churn": "int8"
    },
    parse_dates=["inicio_atencion_utc", "fin_atencion"],
)
# Adecuamos la zona horaria
df["inicio_atencion"] = df["inicio_atencion_utc"].dt.tz_convert(
    "America/Asuncion")
df = df.drop(columns=["inicio_atencion_utc"])
df["fin_atencion"] = df["fin_atencion"].dt.tz_localize("America/Asuncion")

In [ ]:
df_zonas = pd.read_csv("../data/dataset_churn_zona_challenge.csv")

In [ ]:
# Procesamos las coordenadas
df["geometry"] = df["coordenadas_sucursal"].apply(
    lambda x: Point(json.loads(x)["longitud"], json.loads(x)["latitud"])
)
gdf = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")

In [ ]:
gdf_zonas = gpd.GeoDataFrame(
    df_zonas,
    geometry=df_zonas["poligono"].apply(
        lambda x: Polygon(ast.literal_eval(x))),
    crs="EPSG:4326"
)

In [ ]:
# Traemos las zonas
gdf_final = gpd.sjoin(
    gdf,
    gdf_zonas[["zona", "geometry"]],
    how="left",
    predicate="within"
)

In [ ]:
# Hacemos limpieza
dataset = pd.DataFrame(gdf_final.drop(
    columns=["geometry", "index_right", "coordenadas_sucursal"]))

In [ ]:
del df, df_zonas, gdf, gdf_zonas, gdf_final
gc.collect()

In [ ]:
dataset

In [ ]:
def predecir_churn(tipo_asistencia):
    tipo_asistencia = str(tipo_asistencia)
    if tipo_asistencia.lower() == "problema":
        return 1.0  # Siempre devuelve 1 si es un problema
    elif tipo_asistencia.lower() == "reclamo":
        return 0.8 if random.random() < 0.5 else 0  # 50% de probabilidad para reclamos
    else:
        return 0

### Costos y rentabilidad
- Costo de estrategia de retención 20
- Rentabilidad del cliente en corto plazo 8
- Rentabilidad del cliente en largo plazo 50
- Rentabilidad del cliente que se da de baja -2

## EDA

In [ ]:
dataset.describe(include='all').T

In [ ]:
(dataset.isnull().sum() / len(dataset)).plot(
    kind='bar',
    figsize=(10,4),
    title='Proporción de Valores Ausentes',
    ylabel='Proporción'
)
plt.show()

In [ ]:
# Parece que el 01/01/25 se volvió opcional
optional = dataset[dataset["tipo_asistencia"].isnull()]["inicio_atencion"].min()
optional

In [ ]:
# Hay demasiado nulo luego de que se volvió opcional, sería arriesgado usarlo como feature
dataset.query("inicio_atencion >= @optional")[
    "tipo_asistencia"].value_counts(dropna=False, normalize=True)

In [ ]:
dataset["time_diff_min"] = (
    dataset["fin_atencion"] - dataset["inicio_atencion"]
).dt.total_seconds() / 60

In [ ]:
dataset["time_diff_min"].describe()

In [ ]:
# Parece que hay tiempos negativos, esto podría indicar problemas de calidad que habría que revisar...
dataset.query("time_diff_min < 0")

In [ ]:
# Dropeamos el fin de atención,nos vamos a enforcar en el inicio de la atención
dataset = dataset.drop(
    columns=["time_diff_min", "fin_atencion"], errors='ignore')

In [ ]:
# Mi costumbre, para estandarizar el nombre
dataset = dataset.rename(columns={"churn": "target"})

In [ ]:
dataset

In [ ]:
ax = sns.countplot(
    data=dataset,
    x="target"
)
ax.set_xlabel("Target")
ax.set_ylabel("Cantidad")
ax.set_title(
    "Distribución del Target"
)

ax.set_xticks([0, 1])
ax.set_xticklabels(["No Churn", "Churn"])

ax.yaxis.set_major_formatter(StrMethodFormatter("{x:,.0f}"))

for p in ax.patches:
    height = p.get_height()
    percentage = (height / len(dataset)) * 100
    ax.annotate(
        text=f"{percentage:.1f}%",
        xy=(p.get_x() + p.get_width() / 2., height),
        ha="center",
        va="center",
        xytext=(0, 5),
        textcoords="offset points"
    )
plt.show()

In [ ]:
# Solo por si acaso, no sea cosa que un mismo cliente tenga múltiples churns :o
dataset.groupby("cliente_id")["target"].sum(
).sort_values(ascending=False).head(10)

In [ ]:
dataset = dataset.sort_values("inicio_atencion")

In [ ]:
dataset

In [ ]:
df = dataset[["inicio_atencion", "target"]].copy()
df["Año-Mes"] = df["inicio_atencion"].astype(str).str.slice(0, 7)
df["Target"] = df["target"].map({0: "No Churn", 1: "Churn"})

plt.figure(figsize=(12, 6))
ax = sns.countplot(
    data=df,
    x="Año-Mes",
    hue="Target"
)
ax.set_ylabel("Cantidad")
ax.set_title(
    "Distribución del Target por Año y Mes"
)
ax.yaxis.set_major_formatter(StrMethodFormatter("{x:,.0f}"))

plt.show()